In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from tcompress import register_ as reg
from tcompress import algo_cooling as algo
from tcompress import quf
import time
import numpy as np
from tqdm import tqdm
import nlopt
import torch
from torch.nn.utils import parameters_to_vector, vector_to_parameters


import matplotlib.pyplot as plt


In [2]:
reg.reg_complex_svd()

to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_ = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=True)

opt = algo.opt_(progbar=True, max_time="rate:1e9", max_repeats=2**7, optlib="cmaes")


In [3]:
#ITF params
J = 1
h = 3.05
delta = 1.0

In [4]:
Lx, Ly = 3, 4       # lattice dimensions: rows x columns
L = Lx * Ly          # total number of sites
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)
sites = sorted({ (site,) for edge in edges for site in edge})
N = len(sites)

In [5]:
pepo = quf.pepo_identity(Lx, Ly)
pepo.apply_to_arrays(to_backend_c)


In [6]:
depth_total = 3
params = {}

p1 = 1.0 / (2.0 - 2.0**(1/3))
p2 = - (2.0**(1/3)) / (2.0 - 2.0**(1/3))



for depth in range(depth_total):
        if depth ==1:
            phi = -h * 0.5 * p2
            theta = -J * 2 * 0.5 *p2
        else:
            phi = -h * 0.5 * p1
            theta = -J * 2 * 0.5 *p1

        params[f"rx_depth{depth}"] = to_backend_( torch.tensor( phi ).clone().detach() )
        params[f"rzz_depth{depth}"] = to_backend_( torch.tensor( theta ).clone().detach() )


# depth_total = 1
# for depth in range(depth_total):
#         phi = -h * 0.5
#         theta = -J * 2 * 0.5
#         params[f"rx_depth{depth}"] = to_backend_( torch.tensor( phi ).clone().detach() )
#         params[f"rzz_depth{depth}"] = to_backend_( torch.tensor( theta ).clone().detach() )



pepo = algo.skeleten_pepo(params, edges, sites, depth_total=depth_total,contract=True,
                                  to_backend=to_backend_c, Lx=Lx, Ly=Ly)

/home/reza.haghshenas@quantinuum.com/time_compressed_algo/algo_cooling.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=dtype, device=device, requires_grad=requires_grad)


In [7]:
# params_, skeleton = qtn.pack(pepo)
# params_

In [8]:
info_su = qu.load_from_disk(f"store_state/info_su")
info_bp = qu.load_from_disk(f"store_state/info_bp")

pepo_fix = info_su["tn"]
# pepo_fix = info_bp["pepo"]

# pepo_fix = qu.load_from_disk("store/pepo")
# pepo_fix = qu.load_from_disk("store/U_tn")

#pepo_fix.show()

In [9]:
pepo_fix

TensorNetworkGenVector(tensors=12, indices=41)

In [10]:
# norm = (pepo_fix.H & pepo_fix).contract(all, optimize=opt)
# print(complex(norm).real, complex(norm / 2**L).real)

# pepo_fix = pepo_fix * (norm / 2**L)**-0.5
# norm = (pepo_fix.H & pepo_fix).contract(all, optimize=opt)
# print(norm, 2**L, complex(norm / 2**L))


In [11]:
%%time
cost_opts = { "Lx":Lx, "Ly":Ly, "depth_total":depth_total, "opt":opt, "to_backend":to_backend_c  }
cost = algo.cost_function(pepo_fix, params, sites, edges, **cost_opts)
print(  float(cost)   )

0.9496653728613783
CPU times: user 17.8 s, sys: 3.14 s, total: 21 s
Wall time: 1.11 s


In [20]:
params, loss_history = algo.adam_optimize(pepo_fix, params, sites, edges, cost_opts, its_max=10, lr=0.01)

adam: 100%|████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.25it/s, loss=3.043e-01]


In [21]:
params, energy_hist, grad_hist = algo.nlopt_optimize(pepo_fix, params, sites, edges, cost_opts, 
                                                     cost_fn=algo.cost_function, its_max=20
                                                    )

nlopt:  90%|██████████████████████████████████████████████████▍     | 18/20 [00:20<00:02,  1.16s/it, loss=3.024e-01, ||grad||=9.68e-06]


In [14]:
algo.cost_function(pepo_fix, params, sites, edges, **cost_opts)

tensor(0.3024, dtype=torch.float64, grad_fn=<RsubBackward1>)

In [15]:
params

{'rx_depth0': tensor(-2.1417, dtype=torch.float64, requires_grad=True),
 'rzz_depth0': tensor(-0.6255, dtype=torch.float64, requires_grad=True),
 'rx_depth1': tensor(2.7903, dtype=torch.float64, requires_grad=True),
 'rzz_depth1': tensor(0.4917, dtype=torch.float64, requires_grad=True),
 'rx_depth2': tensor(-2.1417, dtype=torch.float64, requires_grad=True),
 'rzz_depth2': tensor(-0.6255, dtype=torch.float64, requires_grad=True)}

In [16]:
# params['rzz_((0, 0), (0, 1))'], params['rzz_((0, 0), (1, 0))']

In [17]:
%%time

energy_log = []
grad_log = []
dtype_ = torch.float64
device = "cpu"

# --- Flatten / unflatten utilities ---
def flatten_params(trainable_params):
    return parameters_to_vector(list(trainable_params.values())).detach().cpu().numpy()

def unflatten_params(trainable_params, x):
    vector_to_parameters(torch.tensor(x, dtype=dtype_, device=device), list(trainable_params.values()))




# --- objective with logging ---
step_counter = {"n": 0}  # mutable container for keeping step count

def objective(x, grad):
    start_time = time.perf_counter()
    step_counter["n"] += 1

    # update torch params
    unflatten_params(params, x)
    elapsed = time.perf_counter() - start_time
    #print("unflatten",  elapsed)

    #start_time = time.perf_counter()
   
    # zero grads
    for p in params.values():
        if p.grad is not None:
            p.grad.zero_()
    elapsed = time.perf_counter() - start_time
    #print(" zero grads", elapsed)

    #start_time = time.perf_counter()

    # compute loss
    loss = algo.cost_function(pepo_fix, params, sites, edges, **cost_opts)

    elapsed = time.perf_counter() - start_time
    #print("cost", elapsed)

    #start_time = time.perf_counter()
    loss.backward()
    elapsed = time.perf_counter() - start_time
    #print("backward", elapsed)

    # flatten grads
    # g = torch.cat([p.grad.reshape(-1) for p in trainable_params.values()])
    # grad_norm = g.norm().item()

    #start_time = time.perf_counter()
    
    # if grad.size > 0:
    #     grad[:] = g.detach().numpy()
    if grad.size > 0:
        flat_grad_tensor = parameters_to_vector([p.grad for p in params.values()])
        grad[:] = flat_grad_tensor.detach().cpu().numpy()
        grad_norm = flat_grad_tensor.norm().item()
    else:
        grad_norm = float('nan')

    elapsed = time.perf_counter() - start_time
    #print("put_grad", elapsed)

    energy_log.append(loss.item())
    grad_log.append(grad_norm)
    # logging
    print(f"Step {step_counter['n']}: Energy = {loss.item():.6f}, Grad norm = {grad_norm:.6e}, t = {elapsed}")

    return loss.item()

# --- run NLopt ---
x0 = flatten_params(params)



its_max = 12

opt_nl = nlopt.opt(nlopt.LD_LBFGS, len(x0))
# opt_nl = nlopt.opt(nlopt.LD_VAR2, len(x0))

opt_nl.set_min_objective(objective)

opt_nl.set_maxeval(its_max)

opt_nl.set_ftol_rel(1e-8)
opt_nl.set_xtol_rel(1e-8)
opt_nl.set_ftol_abs(1e-8)





x_opt = opt_nl.optimize(x0)
final_loss = opt_nl.last_optimum_value()

print("\nOptimization finished.")
print("Final loss =", final_loss)


print(energy_log)
print(grad_log)



Step 1: Energy = 0.302360, Grad norm = 2.019659e-04, t = 0.776634597001248
Step 2: Energy = 0.302360, Grad norm = 2.589117e-03, t = 0.7923399669962237
Step 3: Energy = 0.302360, Grad norm = 5.287004e-05, t = 0.779861584000173
Step 4: Energy = 0.302360, Grad norm = 7.995537e-05, t = 0.7634229419927578
Step 5: Energy = 0.302360, Grad norm = 1.998848e-05, t = 0.7615565290034283

Optimization finished.
Final loss = 0.3023602063860059
[0.30236020795890817, 0.30236043914752875, 0.30236020642958583, 0.30236020645413353, 0.3023602063860059]
[0.00020196589464754883, 0.0025891174497722305, 5.287004448570206e-05, 7.995537157425188e-05, 1.998847549444945e-05]
CPU times: user 1min 44s, sys: 22.1 s, total: 2min 6s
Wall time: 3.89 s


In [18]:
%%time

# --- deps
import time, numpy as np, torch
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from scipy.optimize import minimize

# ----- CONFIG -----
dtype_  = torch.float64
device  = "cpu"
its_max = 4

# Your dict of trainable tensors (all require_grad=True, float dtype)
# params: Dict[str, torch.Tensor]
# cost_function(params) -> scalar torch.Tensor
# -------------------

# --- Flatten / unflatten
def flatten_params(trainable_params):
    return parameters_to_vector(list(trainable_params.values())).detach().cpu().numpy()

def unflatten_params(trainable_params, x):
    vector_to_parameters(torch.tensor(x, dtype=dtype_, device=device),
                         list(trainable_params.values()))

# --- Logs
energy_log, grad_log = [], []
step_counter = {"n": 0}

# --- Value + gradient closure for SciPy
def fun_and_grad(x):
    step_counter["n"] += 1
    t0 = time.perf_counter()

    # load x -> torch params
    unflatten_params(params, x)

    # zero grads
    for p in params.values():
        if p.grad is not None:
            p.grad.zero_()

    # forward
    loss = algo.cost_function(pepo_fix, params, sites, edges, **cost_opts)          # torch scalar
    # backward
    loss.backward()

    # flatten grad (SciPy expects float64 numpy)
    flat_grad = parameters_to_vector([p.grad for p in params.values()]).detach().cpu().numpy()
    grad_norm = float(np.linalg.norm(flat_grad))
    dt = time.perf_counter() - t0

    # logs
    energy_log.append(loss.item())
    grad_log.append(grad_norm)
    print(f"Step {step_counter['n']:5d}: "
          f"InF = {loss.item():.8f}, GradNorm = {grad_norm:.3e}, dt = {dt:.3f}s")

    return loss.item(), flat_grad

# --- SciPy expects separate fun and jac callables, or fun that ignores jac.
#     We wrap fun_and_grad and cache outputs to avoid double evals.
_last_x = {"x": None, "f": None, "g": None}
def fun_only(x):
    if _last_x["x"] is None or not np.array_equal(x, _last_x["x"]):
        f, g = fun_and_grad(x)
        _last_x.update(x=x.copy(), f=f, g=g)
    return _last_x["f"]

def jac_only(x):
    if _last_x["x"] is None or not np.array_equal(x, _last_x["x"]):
        f, g = fun_and_grad(x)
        _last_x.update(x=x.copy(), f=f, g=g)
    return _last_x["g"]

# --- Initial vector
x0 = flatten_params(params)

# --- Run SciPy
res = minimize(
    fun_only,
    x0,
    method="L-BFGS-B",      # or "BFGS"
    jac=jac_only,
    options=dict(
        maxiter=its_max,
        ftol=1e-8,          # function tolerance
        gtol=1e-8,          # gradient tolerance (BFGS); L-BFGS-B uses pgtol via 'pgtol'
        maxcor=20,          # L-BFGS memory (default 10)
        maxls=50,           # line-search steps
        disp=True,
    ),
)

# --- Write back the final params (minimize returns the last x)
unflatten_params(params, res.x)

print("\nOptimization finished.")
print("Success:", res.success, "| Status:", res.status)
print("Final loss:", res.fun)
# logs are in energy_log / grad_log


Step     1: InF = 0.30236021, GradNorm = 1.999e-05, dt = 0.780s
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           20

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.02360D-01    |proj g|=  1.53334D-05


 This problem is unconstrained.


Step     2: InF = 0.97653159, GradNorm = 3.465e-01, dt = 0.766s
Step     3: InF = 0.30236021, GradNorm = 6.576e-05, dt = 0.773s

At iterate    1    f=  3.02360D-01    |proj g|=  4.36271D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      1      3      1     0     0   4.363D-05   3.024D-01
  F =  0.30236020635708560     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

Optimization finished.
Success: True | Status: 0
Final loss: 0.3023602063570856
CPU times: user 1min 2s, sys: 12.8 s, total: 1min 15s
Wall time: 2.33 s
